In [1]:
import sys
sys.path.append('..')

import json
import ollama.client as client

model = 'zephyr'

# response, _ = client.generate(model_name=model,prompt='prompt' )
client.show('zephyr:latest')

{'license': 'MIT License\n\nCopyright (c) [year] [fullname]\n\nPermission is hereby granted, free of charge, to any person obtaining a copy\nof this software and associated documentation files (the "Software"), to deal\nin the Software without restriction, including without limitation the rights\nto use, copy, modify, merge, publish, distribute, sublicense, and/or sell\ncopies of the Software, and to permit persons to whom the Software is\nfurnished to do so, subject to the following conditions:\n\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\n\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\nFITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\nAUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER\nLIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, AR

In [2]:
def graphPrompt(input: str, metadata={}, model="mistral-openorca:latest"):
    if model == None:
        model = "mistral-openorca:latest"

    # model_info = client.show(model_name=model)
    # print( chalk.blue(model_info))

    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include object, entity, location, organization, person, \n"
            "\tcondition, acronym, documents, service, concept, etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input}``` \n\n output: "
    response, _ = client.generate(
        model_name=model, system=SYS_PROMPT, prompt=USER_PROMPT
    )
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        result = None
    return result

In [3]:
r = graphPrompt(
    """
text delimited bweteen ``` \n
text: ```
Mary had a little lamb,
You've heard this tale before;
But did you know she passed her plate, 
And had a little more!
```
""", model='zephyr:latest'
)

[
   {
       "node_1": "Mary",
       "node_2": "lamb",
       "edge": "Mary had a little lamb"
   },
   {
       "node_1": "Mary",
       "node_2": "plate",
       "edge": "Mary passed her plate"
   },
   {
       "node_1": "Mary",
       "node_2": "more",
       "edge": "And had a little more"
   }
]
```

ERROR ### Here is the buggy response:  [
   {
       "node_1": "Mary",
       "node_2": "lamb",
       "edge": "Mary had a little lamb"
   },
   {
       "node_1": "Mary",
       "node_2": "plate",
       "edge": "Mary passed her plate"
   },
   {
       "node_1": "Mary",
       "node_2": "more",
       "edge": "And had a little more"
   }
]
``` 


